<a href="https://colab.research.google.com/github/hassid4luv/hassid4luv/blob/main/Ck_pso_i2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4463 sha256=281b29799961882ef75f19d231796f3e29064edc37c441ea021e6def8aff07e6
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [3]:
import numpy as np
import pandas as pd
from pyswarm import pso
from scipy.integrate import trapz, odeint

# Load data from the Excel file
data = pd.read_excel('CKinetics-2.xlsx')

# Extract the necessary data
heating_rate = data['Heating rate (Change in temperature/min)'].values
time = data['Time (min)'].values
temperature = data['Temperature (Kelvin)'].values
CMLexp = data['m/mo (CML)'].values
MLRexp = data['d(m/mo)dt (MLR)'].values

# Constants
N = 350  # Number of experiments (heating rates)
n = 70  # Number of datasets per experiment
R = 8.314  # Universal gas constant

# Sort time values
sorted_indices = np.argsort(time)
time = time[sorted_indices]

# Define optimization bounds for the parameters
bounds = [
    (0.14, 0.41), (np.exp(16.28) * 60, np.exp(60.72) * 60), (96500, 354000), (0, 7), (-8, 8), (-8, 8), (0.18, 0.57),
    (0.01, 0.04), (np.exp(7.63) * 60, np.exp(39.38) * 60), (50000, 200500), (0, 7), (-8, 8), (-8, 8), (0.09, 0.35),
    (0.09, 0.27), (np.exp(0.69) * 60, np.exp(42.63) * 60), (23000, 325800), (0, 7), (-8, 8), (-8, 8), (0.13, 0.49),
    (0.12, 0.35), (np.exp(7.63) * 60, np.exp(60.72) * 60), (50000, 354000), (0, 7), (-8, 8), (-8, 8), (0.09, 0.57),
    (0.15, 0.44), (np.exp(0.69) * 60, np.exp(60.72) * 60), (23000, 354000), (0, 7), (-8, 8), (-8, 8), (0, 0.60)
]

# Initial guess for the components
initial_guess = [0.27, np.exp(16.28) * 60, 200000, 4, 0, 0, 0.27, 0.03, np.exp(7.63) * 60, 100000, 4, 0, 0, 0.18, 0.23, np.exp(0.69) * 60, 50000, 4, 0, 0, 0.29]

# PSO optimization
lb = [bound[0] for bound in bounds]
ub = [bound[1] for bound in bounds]

# Define the differential equations
def model(Y, t, params):
    Y_c, Y_h, Y_l, Y_e, Y_u, Y_v, Y_chardt = Y
    Y_ho, Y_lo, Y_eo, Y_uo = params[:4]

    # Corrected Y_co calculation
    Y_co = 1 - Y_ho - Y_lo - Y_eo - Y_uo

    epsilon = 1e-10  # Small epsilon to avoid division by zero
    Y_c = max(Y_c, epsilon)
    Y_h = max(Y_h, epsilon)
    Y_l = max(Y_l, epsilon)
    Y_e = max(Y_e, epsilon)
    Y_u = max(Y_u, epsilon)

    dY_cdt = -Y_co * (Y_c / Y_co) ** params[3] * ((Y_co - Y_c) / Y_co) ** params[4] * (-np.log(Y_c / Y_co + epsilon)) ** params[5] * params[1] * np.exp(-params[2] / (R * t))
    dY_hdt = -Y_ho * (Y_h / Y_ho) ** params[10] * ((Y_ho - Y_h) / Y_ho) ** params[11] * (-np.log(Y_h / Y_ho + epsilon)) ** params[12] * params[8] * np.exp(-params[9] / (R * t))
    dY_ldt = -Y_lo * (Y_l / Y_lo) ** params[17] * ((Y_lo - Y_l) / Y_lo) ** params[18] * (-np.log(Y_l / Y_lo + epsilon)) ** params[19] * params[15] * np.exp(-params[16] / (R * t))
    dY_edt = -Y_eo * (Y_e / Y_eo) ** params[24] * ((Y_eo - Y_e) / Y_eo) ** params[25] * (-np.log(Y_e / Y_eo + epsilon)) ** params[26] * params[22] * np.exp(-params[23] / (R * t))
    dY_udt = -Y_uo * (Y_u / Y_uo) ** params[31] * ((Y_uo - Y_u) / Y_uo) ** params[32] * (-np.log(Y_u / Y_uo + epsilon)) ** params[33] * params[29] * np.exp(-params[30] / (R * t))

    dY_vdt = -((1 - params[6]) * dY_cdt + (1 - params[13]) * dY_hdt + (1 - params[20]) * dY_ldt + (1 - params[27]) * dY_edt + (1 - params[34]) * dY_udt)
    dY_chardt = -((params[6]) * dY_cdt + (params[13]) * dY_hdt + (params[20]) * dY_ldt + (params[27]) * dY_edt + (params[34]) * dY_udt)

    return [dY_cdt, dY_hdt, dY_ldt, dY_edt, dY_udt, dY_vdt, dY_chardt]

# Define the objective function
def objective(params):
    Y_ho, Y_lo, Y_eo, Y_uo = params[:4]
    Y_co = 1 - Y_ho - Y_lo - Y_eo - Y_uo

    # Define the initial conditions
    Y0 = [Y_co, 0, 0, 0, 0, 0, 0]  # Initial concentrations for C, H, L, E, U, V, Char

    # Solve the differential equations
    Y_solution = odeint(model, Y0, time, args=(params,))

    # Extract the simulated MLR and CML values
    MLR_mod = -Y_solution[:, 4]  # Extract MLR from the last component (U)
    CML_mod = trapz(-Y_solution[:, 4], x=time)

    # Calculate Phi_1 and Phi_2
    phi1 = np.sum((MLR_mod - MLRexp) ** 2) / np.sum((MLRexp - 1 / len(MLRexp) * np.sum(MLRexp)) ** 2)
    phi2 = np.sum((CML_mod - CMLexp) ** 2) / np.sum((CMLexp - 1 / len(CMLexp) * np.sum(CMLexp)) ** 2)

    return 0.5 * phi1 + 0.5 * phi2

# Use PSO to perform optimization
xopt, fopt = pso(objective, lb, ub, swarmsize=100, maxiter=100)

# Display the optimized parameters and objective function value
param_names = [
    "Y_ho", "Y_lo", "Y_eo", "Y_uo",
    "A_c", "E_c", "n_c", "m_c", "p_c", "v_c",
    "A_h", "E_h", "n_h", "m_h", "p_h", "v_h",
    "A_l", "E_l", "n_l", "m_l", "p_l", "v_l",
    "A_e", "E_e", "n_e", "m_e", "p_e", "v_e",
    "A_u", "E_u", "n_u", "m_u", "p_u", "v_u"
]

print("Optimized Parameters:")
for name, value in zip(param_names, xopt):
    print(f"{name}: {value}")

print("Objective Function Value:", fopt)

<ipython-input-3-deb04653d85c>:56: RuntimeWarning: invalid value encountered in double_scalars
  dY_cdt = -Y_co * (Y_c / Y_co) ** params[3] * ((Y_co - Y_c) / Y_co) ** params[4] * (-np.log(Y_c / Y_co + epsilon)) ** params[5] * params[1] * np.exp(-params[2] / (R * t))
/usr/local/lib/python3.10/dist-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess accuracy requested (tolerances too small). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-deb04653d85c>:83: RuntimeWarning: overflow encountered in square
  phi1 = np.sum((MLR_mod - MLRexp) ** 2) / np.sum((MLRexp - 1 / len(MLRexp) * np.sum(MLRexp)) ** 2)
<ipython-input-3-deb04653d85c>:84: RuntimeWarning: overflow encountered in square
  phi2 = np.sum((CML_mod - CMLexp) ** 2) / np.sum((CMLexp - 1 / len(CMLexp) * np.sum(CMLexp)) ** 2)
<ipython-input-3-deb04653d85c>:60: RuntimeWarning: divide by zero encountered in double_scalars
  dY_udt = -Y_uo * (Y_u / Y_uo

Stopping search: maximum iterations reached --> 100
Optimized Parameters:
Y_ho: 0.15196170218316293
Y_lo: 1.0504318661184978e+28
Y_eo: 96500.0
Y_uo: 5.287897733590659
A_c: 1.3408939043888175
E_c: 8.0
n_c: 0.4002754939303975
m_c: 0.03365527935981284
p_c: 7.597451699202806e+18
v_c: 175242.72313719234
A_h: 2.7782908995054933
E_h: -1.1329077376879653
n_h: -8.0
m_h: 0.09413435450059499
p_h: 0.19685319015631172
v_h: 9.425946925914528e+19
A_l: 220094.78563866028
E_l: 2.59253314982176
n_l: -8.0
m_l: 4.995931513652678
p_l: 0.49
v_l: 0.26737212507586894
A_e: 1.0757161446405905e+28
E_e: 50000.0
n_e: 2.274825458011464
m_e: -0.3921577642118983
p_e: 3.7612800962797
v_e: 0.21418124253157927
A_u: 0.15
E_u: 6.461517573825042e+27
n_u: 301590.8826348681
m_u: 2.5721718881787585
p_u: -0.21914693691321396
v_u: 8.0
Objective Function Value: 1.2816342231309794
